In [ ]:
import requests
import plotly.graph_objects as go
import time
import csv
import os
import pytz
from datetime import datetime
from IPython.display import display, clear_output

# OpenWeatherMap API Key
api_key = "85a5452456eef1609223fe91b8bc4fed"  # Thay thế bằng API key của bạn
base_url = "http://api.openweathermap.org/data/2.5/weather?"

# Thành phố cần lấy dữ liệu thời tiết
city = "Dubai"

# Múi giờ Dubai
dubai_tz = pytz.timezone("Asia/Dubai")

# Danh sách lưu trữ dữ liệu theo thời gian
time_series = []
temperature_series = []

# File CSV để lưu trữ dữ liệu
csv_filename = "temperature_data.csv"
max_records = 200  # Số lần cập nhật tối đa trước khi xóa file

def get_weather_data(city):
    complete_url = f"{base_url}q={city}&appid={api_key}&units=metric"
    response = requests.get(complete_url)
    return response.json()

def save_to_csv(timestamp, temperature):
    global csv_filename
    file_exists = os.path.isfile(csv_filename)

    # Đọc số dòng hiện có để kiểm tra
    if file_exists:
        with open(csv_filename, "r") as f:
            row_count = sum(1 for row in f) - 1  # Trừ dòng tiêu đề
    else:
        row_count = 0

    # Nếu đã đạt giới hạn, xóa file cũ và tạo file mới
    if row_count >= max_records:
        os.remove(csv_filename)
        print("Đã đạt giới hạn lưu trữ, tạo file mới.")
        file_exists = False  # Đánh dấu để tạo file mới

    # Ghi dữ liệu vào file CSV
    with open(csv_filename, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Thời gian (Dubai)", "Nhiệt độ Dubai (°C)"])  # Ghi tiêu đề nếu file mới
        writer.writerow([timestamp, temperature])

def plot_realtime_temperature():
    while True:
        # Lấy thời gian thực của Dubai
        current_time = datetime.now(dubai_tz).strftime("%H:%M:%S")
        time_series.append(current_time)

        data = get_weather_data(city)
        if data["cod"] == 200:
            main = data["main"]
            temperature = main["temp"]
            temperature_series.append(temperature)
        else:
            print(f"Không thể lấy dữ liệu thời tiết của {city}.")
            temperature_series.append(None)

        # Lưu dữ liệu vào CSV
        save_to_csv(current_time, temperature)

        # Giới hạn số điểm dữ liệu để thấy xu hướng dài hơn
        if len(time_series) > 100:  # Lưu trữ 100 điểm (~1.5 giờ nếu cập nhật mỗi 1 phút)
            time_series.pop(0)
            temperature_series.pop(0)

        # Vẽ biểu đồ
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=time_series, y=temperature_series, mode='lines+markers', name='Nhiệt độ Dubai (°C)'))
        fig.add_trace(go.Scatter(x=time_series, y=temperature_series, mode='lines', name='Xu hướng Dubai', line=dict(dash='dot')))

        fig.update_layout(title=f"Nhiệt độ real-time ở Dubai",
                          xaxis_title="Thời gian (Dubai)",
                          yaxis_title="Nhiệt độ (°C)",
                          xaxis=dict(tickangle=-45))

        clear_output(wait=True)
        display(fig)

        time.sleep(60)  # Cập nhật mỗi 1 phút

# Chạy hàm vẽ biểu đồ real-time
plot_realtime_temperature()


Mô tả hoạt động của mã Python theo dõi nhiệt độ real-time tại Dubai
Mã Python trên có chức năng lấy dữ liệu nhiệt độ thực tế của Dubai theo thời gian thực từ OpenWeatherMap API và hiển thị biểu đồ động để trực quan hóa sự thay đổi nhiệt độ theo thời gian. Ngoài ra, dữ liệu nhiệt độ cũng được lưu vào file CSV để phục vụ phân tích sau này.